<a href="https://colab.research.google.com/github/SunSlick2/MoveEmails/blob/main/Online_Archive_Root_to_Inbox_Migrator_(Item_by_Item_Throttling).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import win32com.client
import pythoncom
import logging
import time
from datetime import datetime
import json
import os
import tkinter as tk
from tkinter import messagebox
from win32com.client import Dispatch

# =========================================================================
# THROTTLING CONFIGURATION (Error-Driven Reset)
# This design prioritizes keeping the COM session alive until an error occurs.
# The small batch size prevents hitting hard limits too quickly.
# =========================================================================
MOVE_BATCH_SIZE = 10            # Number of items to move per internal batch.
MOVE_PAUSE_SECONDS = 2          # Short pause after each batch of 10 moves.
ERROR_COOLDOWN_SECONDS = 120    # Long pause upon detecting a throttling error, followed by COM reset.
# =========================================================================

class EmailMigrator:
    # Error codes consistent with server-side throttling/MAPI limits
    SERVER_THROTTLE_ERRORS = [-2147352567, -2147220731]

    def __init__(self):
        self.setup_logging()
        self.migration_report = {
            'start_time': None,
            'end_time': None,
            'source_store': 'Online Archive - ghi.jkl@def.com',
            'destination_folder': 'Inbox',
            'total_successful': 0,
            'total_failed': 0,
            'failed_items': [],
            'config': {
                'batch_size': MOVE_BATCH_SIZE,
                'pause_seconds': MOVE_PAUSE_SECONDS,
                'cooldown_seconds': ERROR_COOLDOWN_SECONDS
            }
        }

    def setup_logging(self):
        """Setup comprehensive logging"""
        log_dir = "migration_logs"
        os.makedirs(log_dir, exist_ok=True)
        log_filename = os.path.join(log_dir, f'archive_migrator_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

        # Configure logging to file and console
        logging.basicConfig(
            filename=log_filename,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(levelname)s: %(message)s')
        console.setFormatter(formatter)
        logging.getLogger().addHandler(console)
        logging.info(f"Logging to: {log_filename}")

    def find_store_by_display_name(self, namespace, display_name):
        """Finds an Outlook store (mailbox/archive) by its display name."""
        for store in namespace.Stores:
            if store.DisplayName == display_name:
                return store
        return None

    def move_small_batch(self, namespace, source_folder, destination_folder):
        """
        Attempts to move a small batch of items from the source folder.
        It moves the item at index 1 repeatedly.
        Returns the number of successful moves in this batch.
        """
        moved_count_session = 0
        items = source_folder.Items

        logging.debug(f"Source folder current item count estimate: {items.Count}")

        for i in range(MOVE_BATCH_SIZE):
            try:
                # 1. Check if the folder is empty
                if items.Count == 0:
                    logging.info("Source folder is now empty. Batch complete.")
                    return moved_count_session # Migration finished

                # 2. Access the item at index 1 (the first item).
                # This is crucial for collections that are being modified.
                item = items.Item(1)

                # Check for item class (olMail)
                if getattr(item, 'Class', None) != 43:
                    # If it's not a mail item (e.g., meeting request or other), skip it.
                    # We move it to the destination if possible to clean the root,
                    # but only count olMail items as successful migration targets.
                    subject = getattr(item, 'Subject', f'Non-Mail Item Class {getattr(item, "Class", "N/A")}')
                    item.Move(destination_folder)
                    logging.warning(f"SKIPPED NON-MAIL ITEM: {subject[:70]}...")
                    continue

                subject = getattr(item, 'Subject', 'N/A')

                # 3. Perform the move
                item.Move(destination_folder)

                moved_count_session += 1
                self.migration_report['total_successful'] += 1

                logging.info(f"SUCCESS ({self.migration_report['total_successful']}): {subject[:70]}...")

            except pythoncom.com_error as com_err:
                error_code = com_err.args[0]
                error_details = str(com_err)

                # If it's a known throttling error, we need a full cooldown and reset
                if error_code in self.SERVER_THROTTLE_ERRORS or (com_err.args[2] and com_err.args[2][5] in self.SERVER_THROTTLE_ERRORS):
                    error_type = "Server Throttling Detected"
                    logging.error(f"{error_type} after {moved_count_session} successful moves in this batch.")
                else:
                    # Log other COM errors as failures, but still break the batch
                    error_type = f"General COM Error: {error_code}"
                    logging.error(f"Failed to move item '{subject[:70]}...': {error_details}")
                    self.migration_report['total_failed'] += 1
                    self.migration_report['failed_items'].append({
                        'subject': subject,
                        'error': error_details,
                        'type': error_type
                    })

                # Stop this batch immediately and signal the need for a full reset/cooldown
                return moved_count_session, True # Return moves count and signal error

            except Exception as e:
                # Handle general Python exceptions (e.g., if item is suddenly invalid)
                error_details = str(e)
                error_type = "Python/General Runtime Error"
                logging.error(f"Failed to move item '{subject[:70]}...': {error_details}")
                self.migration_report['total_failed'] += 1
                self.migration_report['failed_items'].append({
                    'subject': subject,
                    'error': error_details,
                    'type': error_type
                })
                # Stop this batch immediately and signal the need for a full reset/cooldown
                return moved_count_session, True # Return moves count and signal error

            # Apply internal pause after each MOVE_BATCH_SIZE is completed successfully
            if (i + 1) % MOVE_BATCH_SIZE == 0:
                logging.info(f"Completed internal batch of {MOVE_BATCH_SIZE} moves. Pausing {MOVE_PAUSE_SECONDS} seconds...")
                time.sleep(MOVE_PAUSE_SECONDS)

        # If the loop completed the full batch without error
        return moved_count_session, False

    def run_migration(self):
        """
        Main function using a single long-running COM session, resetting only on error.
        No initial full-folder enumeration is performed.
        """
        logging.info("Starting Online Archive Root to Inbox migration with item-by-item throttling.")
        self.migration_report['start_time'] = datetime.now().isoformat()

        is_initialized = False
        migration_in_progress = True

        while migration_in_progress:
            outlook = None
            try:
                # 1. Initialize COM and Outlook objects
                if not is_initialized:
                    logging.info("-" * 50)
                    logging.info("Starting new COM session...")
                    pythoncom.CoInitialize()
                    is_initialized = True

                outlook = Dispatch("Outlook.Application")
                namespace = outlook.GetNamespace("MAPI")

                # 2. Locate the Archive Store and Folders
                archive_store = self.find_store_by_display_name(namespace, self.migration_report['source_store'])
                if not archive_store:
                    messagebox.showerror("Initialization Error", f"Online Archive '{self.migration_report['source_store']}' not found. Ensure it is connected.")
                    raise ValueError("Archive store not found.")

                source_folder = archive_store.GetRootFolder()
                destination_folder = source_folder.Folders['Inbox']

                # Initial check for items and confirmation (only on the very first run)
                if self.migration_report['total_successful'] == 0 and self.migration_report['total_failed'] == 0:
                    initial_count_estimate = source_folder.Items.Count

                    if initial_count_estimate == 0:
                        logging.info("Initial check shows source folder is empty. Exiting.")
                        break

                    if not messagebox.askyesno("Confirm Migration",
                        f"You are about to MOVE items from the Archive Root "
                        f"to the Inbox of '{archive_store.DisplayName}'.\n\n"
                        f"Initial item count estimate: {initial_count_estimate}\n"
                        f"This process will use error-driven resets.\n\n"
                        f"Do you want to proceed?"
                    ):
                        logging.warning("Migration cancelled by user.")
                        break


                # 3. Main migration loop (moves one small batch at a time)
                while True:
                    # Check if the source folder is empty before attempting a move
                    if source_folder.Items.Count == 0:
                        logging.info("Source folder is empty. Migration complete.")
                        migration_in_progress = False
                        break

                    moves_in_batch, error_occurred = self.move_small_batch(namespace, source_folder, destination_folder)

                    if error_occurred:
                        logging.error(f"Migration session hit an error after moving {self.migration_report['total_successful']} total items. Forcing reset and cool-down.")
                        break # Break inner loop, trigger reset/cooldown

                    if not migration_in_progress:
                        break # Break if folder became empty in the batch

                    # If no error, continue to the next batch in the same session
                    logging.info(f"Successfully moved {moves_in_batch} items in batch. Continuing in current session...")

                # 4. If migration completed successfully, break the outer loop
                if not migration_in_progress:
                    break

                # 5. If we broke due to an error, we execute the aggressive cool-down
                del outlook
                outlook = None
                pythoncom.CoUninitialize()
                is_initialized = False

                logging.info(f"Commencing long cool-down for {ERROR_COOLDOWN_SECONDS} seconds...")
                time.sleep(ERROR_COOLDOWN_SECONDS)
                logging.info("Cool-down finished. Attempting to restart session.")

            except Exception as e:
                # Catch any critical exception (e.g., Outlook crash, store suddenly offline)
                logging.error(f"Critical error during session loop: {e}", exc_info=True)
                messagebox.showerror("Migration Error", f"A critical error occurred. Check logs for details: {e}")

                if outlook:
                    del outlook

                # Since the exception might have occurred before CoUninitialize(), call it conditionally
                if is_initialized:
                    pythoncom.CoUninitialize()

                migration_in_progress = False
                break

        # Final Report Generation
        self.generate_report()
        logging.info("\nMigration process finished.")

        return self.migration_report['total_failed'] == 0

    def generate_report(self):
        """Generate a comprehensive migration report"""
        report_dir = "migration_reports"
        os.makedirs(report_dir, exist_ok=True)
        report_filename = os.path.join(report_dir, f'migration_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')

        self.migration_report['end_time'] = datetime.now().isoformat()

        # Total attempted is now equal to successful + failed, as we don't pre-count total items
        total_attempted = self.migration_report['total_successful'] + self.migration_report['total_failed']
        self.migration_report['total_attempted'] = total_attempted

        if self.migration_report['start_time']:
            self.migration_report['duration_seconds'] = (
                datetime.fromisoformat(self.migration_report['end_time']) -
                datetime.fromisoformat(self.migration_report['start_time'])
            ).total_seconds()
        else:
            self.migration_report['duration_seconds'] = 0

        self.migration_report['success_rate'] = (
            self.migration_report['total_successful'] / total_attempted * 100
        ) if total_attempted > 0 else 0

        with open(report_filename, 'w') as f:
            json.dump(self.migration_report, f, indent=2, default=str)

        logging.info(f"Migration report saved to: {report_filename}")

        # Print summary to console
        print(f"\n=== MIGRATION SUMMARY ===")
        print(f"Start Time: {self.migration_report['start_time']}")
        print(f"End Time: {self.migration_report['end_time']}")
        print(f"Duration: {self.migration_report['duration_seconds']:.2f} seconds")
        print(f"Successful Moves: {self.migration_report['total_successful']}")
        print(f"Failed Moves: {self.migration_report['total_failed']}")
        print(f"Success Rate: {self.migration_report['success_rate']:.2f}%")

        if self.migration_report['failed_items']:
            print(f"\n=== FAILED ITEMS ({len(self.migration_report['failed_items'])}) ===")
            print("Showing first 5 failures (check log file for all details):")
            for i, failed_item in enumerate(self.migration_report['failed_items'][:5], 1):
                print(f"{i}. Subject: {failed_item['subject'][:70]}...")
                print(f"   Error Type: {failed_item['type']}")
                print(f"   Error Details: {failed_item['error'][:70]}...")
                print("-" * 30)

# Usage
if __name__ == "__main__":
    # Initialize tkinter for the messagebox, though the window itself remains hidden
    root = tk.Tk()
    root.withdraw()

    print("\n" + "=" * 60)
    print("  Online Archive Root to Inbox Migrator (Error-Driven Throttling)  ")
    print("=" * 60)
    print("This tool attempts to move items one-by-one, resetting only when a")
    print(f"server error is encountered, followed by a {ERROR_COOLDOWN_SECONDS} second cool-down.")
    print("Manual monitoring of the source folder is required for completion.")
    print("Please ensure Outlook is running and the Online Archive is connected.")
    print("\n" + "=" * 60)

    migrator = EmailMigrator()
    success = migrator.run_migration()

    if success:
        messagebox.showinfo("Migration Complete", "Migration completed successfully with no recorded failed moves!")
        print("\n✅ Migration completed successfully with no recorded failed moves!")
    else:
        messagebox.showwarning("Migration Finished", f"Migration completed with {migrator.migration_report['total_failed']} failures. Check logs for details.")
        print("\n⚠️ Migration completed with failures. Check 'migration_logs' and 'migration_reports' folders for details.")